# Prototipo simulador parte 2: Intercambio gaseoso

In [2]:
# Librerías
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Estilo
plt.style.use('seaborn-v0_8-whitegrid')

In [1]:
def modelo_dos_compartimentos(t, y, P_aw_func, R1, E1, R2, E2):
    """
    Define el sistema de EDOs para el modelo de dos compartimentos.

    Argumentos:
    t: tiempo actual
    y: arreglo con los valores actuales de las variables de estado [V1, V2].
    P_aw_func: una función que devuelve la presión aplicada (Paw) en el tiempo t.
    R1, E1, R2, E2: parámetros de resistencia y elastancia de los compartimentos.

    Retorna:
    Una lista con las derivadas de cada variable de estado: [dV1/dt, dV2/dt].
    """
    V1, V2 = y
    P_aw = P_aw_func(t)

    # Ecuaciones diferenciales del modelo, reorganizadas para dV/dt
    # dV1/dt = (P_aw - E1 * V1) / R1
    # dV2/dt = (P_aw - E2 * V2) / R2

    dV1_dt = (P_aw - E1 * V1) / R1
    dV2_dt = (P_aw - E2 * V2) / R2

    return [dV1_dt, dV2_dt]

In [3]:
def modelo_vcv(t, y, flujo_insp, T_insp, PEEP, R1, E1, R2, E2):
    """
    Define el sistema de EDOs para el modo VCV.
    Durante la inspiración, el flujo total es constante.
    Durante la espiración, la presión es constante (PEEP).
    """
    V1, V2 = y

    # ---- Lógica Inspiratoria ----
    numerador_insp = flujo_insp + (E1 * V1 / R1) + (E2 * V2 / R2)
    denominador_insp = (1 / R1) + (1 / R2)
    P_aw_insp = numerador_insp / denominador_insp
    dV1_dt_insp = (P_aw_insp - E1 * V1) / R1
    dV2_dt_insp = (P_aw_insp - E2 * V2) / R2

    # ---- Lógica Espiratoria Vectorizada ----
    P_aw_esp = PEEP
    """dV1_dt_esp = (P_aw_esp - E1 * V1) / R1
    dV2_dt_esp = (P_aw_esp - E2 * V2) / R2"""
    dV1_dt_esp = (-E1 * V1) / R1
    dV2_dt_esp = (-E2 * V2) / R2

    # ---- Combinar usando np.where ----
    es_inspiracion = t < T_insp
    dV1_dt = np.where(es_inspiracion, dV1_dt_insp, dV1_dt_esp)
    dV2_dt = np.where(es_inspiracion, dV2_dt_insp, dV2_dt_esp)

    return [dV1_dt, dV2_dt]